In [1]:
import pandas as pd
import numpy as np
import array
import json
import time

import tensorflow as tf
import tensorflow.keras.preprocessing.text as kpt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from sklearn.model_selection import StratifiedKFold

Para este ejemplo se realizará la predicción utilizando solo el texto del dataset original.

In [2]:
train_df = pd.read_csv('../Data/train_processed.csv')
test_df = pd.read_csv('../Data/test_processed.csv')

In [3]:
# Creo los datos y los labels.
train_x = np.asarray(train_df['text_clean'])
train_y = np.asarray(train_df['target'])

test_x = np.asarray(test_df['text_clean'])

In [4]:
# Solo trabajo con las 3000 palabras mas populares del set de datos.
palabras_maximas = 3000

# Defino una función que me procese los datos.
def procesarDatos(arrayDeTextos):
    # Creo el tokenizador.
    tokenizador = Tokenizer(num_words = palabras_maximas)

    # Le paso los tweets al tokenizer.
    tokenizador.fit_on_texts(arrayDeTextos)

    # Lista de palabras y de indices que me da el tokenizer.
    diccionario = tokenizador.word_index    

    tweetsComoIndicesDePalabras = []

    # Función que convierte los tweets a un vector de indices.
    for tweet in arrayDeTextos:
        indicesDePalabrasDelTweet = [diccionario[palabra] for palabra in kpt.text_to_word_sequence(tweet)]
        tweetsComoIndicesDePalabras.append(indicesDePalabrasDelTweet)
    
    # Paso a array de numpy.
    tweetsComoIndicesDePalabras = np.asarray(tweetsComoIndicesDePalabras)

    # Creo una matriz del tipo one-hot encoding.
    matriz_x = tokenizador.sequences_to_matrix(tweetsComoIndicesDePalabras, mode='binary')

    return matriz_x

In [5]:
train_matriz_x = procesarDatos(train_x)
test_matriz_x = procesarDatos(test_x)

# La lista de labels es una lista de categorias: 1 tweet real; 0 tweet no real.
train_y = tf.keras.utils.to_categorical(train_y, 2)

In [6]:
# Creo el modelo sequencial de tensorflow utilizando keras.

# HIPERPARÁMETROS A TENER EN CUENTA:
# 1 - Cantidad de palabras máximas.
# 2 - Número de salida del primer layer.
# 3 - Función de activación del primer layer.
# 4 - El porcentage de desactivación del primer dropout.
# 5 - Número de salida del segundo layer.
# 6 - Función de activación del segundo layer.
# 7 - El porcentage de desactivación del segundo dropout.
# 8 - Función de activación del layer final.
# 9 - La función loss del modelo.
# 10 - La función optimizer del modelo.

def crearModelo():
    # Red neuronal que es un stack de layer ejecutados en orden.
    modelo = Sequential() 

    # El primer y último layer son de los más importantes ya que son la entrada y la salida.
    # El primer layer tiene entrada de 3000 que es la cantidad máxima de palabras.
    # 512 es la salida de este layer y la función de activación/maximización es relu.
    modelo.add(Dense(512, input_shape=(palabras_maximas,), activation='relu'))

    # Desactivo neuronas con el fin de no caer en overfitting.
    modelo.add(Dropout(0.2))

    # Capa intermedia que tiene como salida 256 neuronas y la función de activación es sigmoid.
    modelo.add(Dense(256, activation='sigmoid'))

    # Desactivo neuronas con el fin de no caer en overfitting.
    modelo.add(Dropout(0.2))

    # Layer de salida, tiene solo 2 que la cantidad de posibilidades que tiene el target.
    # Su función de activación es softmax.
    modelo.add(Dense(2, activation='softmax'))

    # Finalmente compilo el modelo.
    # Cómo métrica le pido que me devuelva el accuracy, la precisión y el recall 
    # para poder calcular con estas últimas dos la puntuación F1.
    modelo.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    
    return modelo

In [7]:
# Entreno el modelo.

# HIPERPARÁMETROS A TENER EN CUENTA:
# 1 - Batch_size.
# 2 - Epochs.
# 3 - Split.

# Entro el modelo tomando los datos de a 32 registros.
# Hago 20 iteraciones.
# Valido con el 20% de los datos.
modelo = crearModelo()
historia = modelo.fit(train_matriz_x, train_y, batch_size = 32, epochs = 20, validation_split = 0.33)

Epoch 1/20
160/160 [==============================] - 3s 18ms/step - loss: 0.6611 - accuracy: 0.6214 - precision: 0.6214 - recall: 0.6214 - val_loss: 0.4801 - val_accuracy: 0.7676 - val_precision: 0.7676 - val_recall: 0.7676
Epoch 2/20
160/160 [==============================] - 2s 12ms/step - loss: 0.3102 - accuracy: 0.8770 - precision: 0.8770 - recall: 0.8770 - val_loss: 0.4815 - val_accuracy: 0.7791 - val_precision: 0.7791 - val_recall: 0.7791
Epoch 3/20
160/160 [==============================] - 2s 13ms/step - loss: 0.1861 - accuracy: 0.9363 - precision: 0.9363 - recall: 0.9363 - val_loss: 0.5732 - val_accuracy: 0.7549 - val_precision: 0.7549 - val_recall: 0.7549
Epoch 4/20
160/160 [==============================] - 2s 14ms/step - loss: 0.1185 - accuracy: 0.9620 - precision: 0.9620 - recall: 0.9620 - val_loss: 0.6317 - val_accuracy: 0.7481 - val_precision: 0.7481 - val_recall: 0.7481
Epoch 5/20
160/160 [==============================] - 2s 14ms/step - loss: 0.1032 - accuracy: 0.9691

In [8]:
predicciones = modelo.predict(test_matriz_x)
categorias = array.array('i')

for prediccion in predicciones:
    categorias.append(np.argmax(prediccion))
    
# Genero un dataframe de pandas y lo guardo.
test_prediccion = test_df['id'].to_frame()
test_prediccion['target'] = pd.Series(categorias)
test_prediccion.to_csv('tensorFlow.csv', index = False)

precision = historia.history[modelo.metrics_names[2]][-1]
recall = historia.history[modelo.metrics_names[3]][-1]
f1 = precision * recall / (precision + recall)
print('El puntaje logrado por este algoritmo fue de: ', f1)

El puntaje logrado por este algoritmo fue de:  0.48852941393852234


### Optimizacion del clasificador de texto básico
En este notebook se buscaran parámetros que optimicen al clasificador. <br>
Recordemos que solo se utiliza el texto para realizar la clasificación. <br>
Como fuimos nombrando a lo largo de este notebook los siguientes son los parámetros que podríamos optimizar:
- 1 - Cantidad de palabras máximas.
- 2 - Número de salida del primer layer.
- 3 - Función de activación del primer layer.
- 4 - El porcentage de desactivación del primer dropout.
- 5 - Número de salida del segundo layer.
- 6 - Función de activación del segundo layer.
- 7 - El porcentage de desactivación del segundo dropout.
- 8 - Función de activación del layer final.
- 9 - La función loss del modelo.
- 10 - La función optimizer del modelo.
- 11 - Batch_size.
- 12 - Epochs.
- 13 - Split.<br>

Sin embargo para este ejemplo solo nos vamos a concentrar en optimizar aquellos parámetros que son necesarios para realizar el fit, para ello se hará una especie de gridsearch con los parámetros batch_size, epochs y validation_split.<br> Para cada parámetro se dejaran fijos con los valores default los demás y se buscara el valor que optimice el fit.<br>
Los valores default para los parámetros a optimizar son:<br>
- batch_size = 32.
- epochs = 1.
- validation_split = 0.0.

Una vez que se obtengan los parámetros que maximizan este train, se hará predicciones sobre el test.
Claramente esto dará un overfitting bastante marcado ya que se esta sobreentrenando el modelo con los datos ingresados.

In [9]:
# Optimización del parámetro validation_split.
def optimizacionValidacion(batch_size, epochs):
    validacion = 0.01
    validacionDelta = 0.01

    mejor_resultado = 0
    mejor_validacion = 0.1
    
    modelo_val_opt = crearModelo()

    while validacion < 1.0:
        print('#################################### Validación: ', validacion, ' ####################################')
        historia_val_opt = modelo_val_opt.fit(train_matriz_x, train_y, batch_size = batch_size, epochs = epochs, validation_split = validacion, verbose = 0)
    
        precision_val_opt = historia_val_opt.history[modelo_val_opt.metrics_names[2]][-1]
        recall_val_opt = historia_val_opt.history[modelo_val_opt.metrics_names[3]][-1]
        f1_val_opt = 2 * (precision_val_opt * recall_val_opt / (precision_val_opt + recall_val_opt))
        acc_val_opt = historia_val_opt.history[modelo_val_opt.metrics_names[1]][-1]
        
        print('Validación: ', validacion, '; Puntaje: ', f1_val_opt)
    
        if f1_val_opt > mejor_resultado:
            mejor_resultado = f1_val_opt
            mejor_validacion = validacion
            print('Mejor resultado para el split: ', validacion)
        
        if acc_val_opt == 1:
            return mejor_resultado, mejor_validacion
        
        validacion = validacion + validacionDelta
    
    return mejor_resultado, mejor_validacion

In [10]:
# Optimización del parámetro epochs.
def optimizacionEpochs(batch_size, validation_split):
    epochs_value = 1
    epochsDelta = 1
    
    mejor_resultado = 0
    mejor_epochs = 1
    
    modelo_epochs_opt = crearModelo()
    
    while epochs_value <= 25:
        print('#################################### Epochs: ', epochs_value, ' ####################################')
        historia_epochs_opt = modelo_epochs_opt.fit(train_matriz_x, train_y, batch_size = batch_size, epochs = epochs_value, validation_split = validation_split, verbose = 0)
        
        precision_epochs_opt = historia_epochs_opt.history[modelo_epochs_opt.metrics_names[2]][-1]
        recall_epochs_opt = historia_epochs_opt.history[modelo_epochs_opt.metrics_names[3]][-1]
        f1_epochs_opt = 2 * (precision_epochs_opt * recall_epochs_opt / (precision_epochs_opt + recall_epochs_opt))
        acc_epochs_opt = historia_epochs_opt.history[modelo_epochs_opt.metrics_names[1]][-1]
        
        print('Epochs: ', epochs_value, '; Puntaje: ', f1_epochs_opt)
        
        if f1_epochs_opt > mejor_resultado:
            mejor_resultado = f1_epochs_opt
            mejor_epochs = epochs_value
            print('Mejor resultado para el Epochs: ', epochs_value)
        
        if acc_epochs_opt == 1:
            return mejor_resultado, mejor_epochs
        
        epochs_value = epochs_value + epochsDelta
    
    return mejor_resultado, mejor_epochs

In [11]:
# Optimización del parámetro bach_size.
def optimizacionBatchSize(epochs, validation_split):
    batch_size = 1
    batch_size_delta = 2
    
    mejor_resultado = 0
    mejor_batch = 1
    
    modelo_batch_opt = crearModelo()
    
    while batch_size <= len(train_y):
        print('#################################### Batch: ', batch_size, ' ####################################')
        historia_batch_opt = modelo_batch_opt.fit(train_matriz_x, train_y, batch_size = batch_size, epochs = epochs, validation_split = validacion, verbose = 0)
        
        precision_batch_opt = historia_batch_opt.history[modelo_batch_opt.metrics_names[2]][-1]
        recall_batch_opt = historia_batch_opt.history[modelo_batch_opt.metrics_names[3]][-1]
        f1_batch_opt = 2 * (precision_batch_opt * recall_batch_opt / (precision_batch_opt + recall_batch_opt))
        acc_batch_opt = historia_batch_opt.history[modelo_batch_opt.metrics_names[1]][-1]
        
        print('Batch: ', batch_size, '; Puntaje: ', f1_batch_opt)
        
        if f1_batch_opt > mejor_resultado:
            mejor_resultado = f1_batch_opt
            mejor_batch = batch_size
            print('Mejor resultado para el Batch: ', batch_size)
            
        if acc_batch_opt == 1:
            return mejor_resultado, batch_size
        
        if batch_size > 100:
            batch_size = batch_size + 100
        else:
            batch_size = batch_size * batch_size_delta
    
    return mejor_resultado, batch_size
        

In [ ]:
# Empiezo el timer.
tiempo_comienzo = time.time()

# Valores default.
default_batch_size = 32
default_epochs = 1
default_split = 0.0

mejor_resultado = 0
mejor_batch_size = 32
mejor_epochs = 1
mejor_split = 0.0

# Optimizaciones.
# 1 - Con los demás parámetros en default se buscará el mejor batch_size, epochs y validation_split.
# Con esto obtengo los primeros valores de batch_size, epochs y validation_split para empezar a trabajar.
mejor_resultado_val, validacion = optimizacionValidacion(default_batch_size, default_epochs)
mejor_split = validacion
    
mejor_resultado_epochs, epochs = optimizacionEpochs(default_batch_size, default_split)
mejor_epochs = epochs
    
mejor_resultado_batch, batch_size = optimizacionBatchSize(default_epochs, default_split)
mejor_batch_size = batch_size


#################################### Validación:  0.01  ####################################
Validación:  0.01 ; Puntaje:  0.743763267993927
Mejor resultado para el split:  0.01
#################################### Validación:  0.02  ####################################
Validación:  0.02 ; Puntaje:  0.8644772171974182
Mejor resultado para el split:  0.02
#################################### Validación:  0.03  ####################################
Validación:  0.03 ; Puntaje:  0.9135969877243042
Mejor resultado para el split:  0.03
#################################### Validación:  0.04  ####################################
Validación:  0.04 ; Puntaje:  0.951286256313324
Mejor resultado para el split:  0.04
#################################### Validación:  0.05  ####################################


In [ ]:
# Con los datos obtenidos hacemos las predicciones.
modelo_predictor = crearModelo()
modelo_predictor.fit(train_matriz_x, train_y, batch_size = mejor_batch_size, epochs = mejor_epochs, validation_split = mejor_split)
modelo_predic = modelo_predictor.predict(test_matriz_x)
modelo_categorias = array.array('i')

for prediccion in modelo_predic:
    modelo_categorias.append(np.argmax(prediccion))
    
# Genero un dataframe de pandas y lo guardo.
modelo_predic_df = test_df['id'].to_frame()
modelo_predic_df['target'] = pd.Series(modelo_categorias)
modelo_predic_df.to_csv('tensorFlow-opt.csv', index = False)
